In [1]:
import  requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
my_list = []
city_list = ["bangalore","mumbai","hyderabad","pune","goa","gokarna"]
for city in city_list:
    url = "https://highape.com/" + city + "/special/nye/best-new-year-2019-events-and-parties"
    page = requests.get(url).text
    page_data = BeautifulSoup(page,"lxml")
    div = page_data.find_all("div",class_ = "col-md-4 col-sm-4 col-lg-4 col-xs-12 much_card_padding event_div") 
    for links in div:
        event_page_url = links.find("a",class_ = "open_event_page")["href"]                                 # single div
        event_page = requests.get(event_page_url).text
        event_page_data = BeautifulSoup(event_page,"lxml")
        title = event_page_data.find("h1", class_ = "event-name-heading").text                          # title
        location = event_page_data.find("h2", class_ = "venue-name-details-event").text                 
        location_lst = location.split("·")
        loc = location_lst[1].strip()                                                         # location
        city = location_lst[2].strip()                                                       # city
        venue = location_lst[0].strip()
        venue_lst = venue.split("-")
        final_venue = venue_lst[1].strip()                                                  #venue
        highlights = event_page_data.find("span", class_ = "details").text
        highlights = highlights.replace("\n","")
        highlights = highlights.replace("·",", ")                                   #highlights
        first_div = event_page_data.find("div", class_ = "first_div")
        divs_in_first_div = first_div.find_all("div", class_ = "quick_look_divs")
        for items in range(0,len(divs_in_first_div)):
            if items == 1:
                schedule = divs_in_first_div[items].text.strip()                       #schedule
            elif items == 2:
                price = divs_in_first_div[items].text
                price = price.replace("Check all prices >","").strip()                # price
        second_div = event_page_data.find("div", class_ = "second_div")
        divs_in_second_div = second_div.find_all("div", class_ = "quick_look_divs")
        if len(divs_in_second_div) == 1:                                 # Extracting artists and organised by
            for items in divs_in_second_div:
                organised_by = items.text.strip()
                organised_by = organised_by.replace("Organised by: ","")
                final_artists = "-"
        elif len(divs_in_second_div) == 2:
            for itemss in range(0,len(divs_in_second_div)):
                if itemss == 0:
                    artists = divs_in_second_div[itemss].text
                    artists = artists.replace("Artists: ","")
                    artist_list = artists.split(",")
                    final_artists = ""
                    for x in artist_list:
                        y = x.strip()
                        final_artists = final_artists + y
                        final_artists = final_artists + ", "


                elif itemss == 1:
                    organised_by = divs_in_second_div[itemss].text.strip()
                    organised_by = organised_by.replace("Organised by: ","")    

        add = event_page_data.find("div",class_ = "col-md-9 col-sm-9 col-lg-9 address")
        address = add.find("p").text                                                              # address
        spc = event_page_data.find("div", class_ = "third_div").text.strip()
        note = spc.replace("Special Details: ","")                                             # note
        google_link = event_page_data.find("div",class_ = "col-md-8 col-sm-8 col-lg-8 green_text")
        gmap_link = google_link.find("a", target = "_blank")["href"]                                     # googlemap link
        #booking_link = event_page_data.find("a", id = "book_thru_me")["href"]                           # booking link
        my_dict = {}
        my_dict["Title"] = title
        my_dict["Location"] = loc
        my_dict["City"] = city
        my_dict["Venue"] = final_venue
        my_dict["Highlights"] = highlights
        my_dict["Schedule"] = schedule
        my_dict["Price"] = price
        my_dict["Organised by"] = organised_by
        my_dict["Artists"] = final_artists
        my_dict["Address"] = address
        my_dict["Note"] = note
        my_dict["Googlemap Link"] = gmap_link
        #my_dict["Book here"] = booking_link
        my_list.append(my_dict)
df = pd.DataFrame(my_list)

In [4]:
df

,Title,Location,City,Venue,Highlights,Schedule,Price,Organised by,Artists,Address,Note,Googlemap Link
0,Moonlit New Year Eve Party 2023,Whitefield,Bangalore,Virginia Mall,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 05:00 PM - Sunday, 01 Jan 12:...",499 - 1499 INR,Oxygen9events,"Krish Hart, DJ Prackze, DJ Prashanth, DJ Nitin...","Virginia Mall, Varthur Main Rd, Palm Meadows, ...",Coupon: Get 5% OFF. Use Code MOON5 \n\n\n\nTh...,"https://www.google.com/maps/place/12.9572489,7..."
1,New Year Bash 2023 At Nolimmits Lounge And Clu...,Brigade Road,Bangalore,NoLimmits Lounge and Club,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 08:30 PM - Sunday, 01 Jan 12:...",3150 - 7350 INR,Nolimmits Lounge & Club,"Dj Esha, DJ VCKY, DJ VeeJay,","Bengaluru, Ashok Nagar, Bangalore Urban, Karna...",Coupon: Get 10% OFF. Use Code NOLIMMITS10 \n\...,"https://www.google.com/maps/place/12.970303,77..."
2,Soiree NYE 2023,Whitefield,Bangalore,Bengaluru Marriott Hotel Whitefield,"Dj Night, Band Performance, Bollywood Night, F...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 01:...",2006 - 11799 INR,Marriot Bangalore,-,"75 EPIP Area, 8th Rd, EPIP Zone, Whitefield, B...",Coupon: Get 10% OFF. Use Code SOIREE10 \n\n\n...,"https://www.google.com/maps/place/12.979255,77..."
3,Baadshaah NYE 2023 Carnival,marathahalli,Bangalore,The Big Baadshaah,"Dj Night, Bollywood Night, Food & Drinks, New ...","Saturday, 31 Dec 07:00 PM - Sunday, 01 Jan 12:...",2399 - 5999 INR,THE BIG BAADSHAAH,"DJ Savanth,","88, 2nd Floor, Dr Puneeth Rajkumar Rd, near Mo...",Coupon: Get 10% OFF. Use Code CARNIVAL10 \n\n...,"https://www.google.com/maps/place/12.9484311,7..."
4,Celestial Supernova - NYE 2023 (Sarjapur),Kaikondrahalli,Bangalore,Big Pitcher Sarjapur Road,"Dj Night, Bollywood Night, Concert, Offers, Fi...","Saturday, 31 Dec 06:30 PM - Sunday, 01 Jan 12:...",0 - 7488 INR,FOODALE INDIA PVT LTD,"DJ Rohith, Dj Shugi Born, DJ Pruthvi, DJ Tony,...","Doddakannelli, 116/9, Ambalipura - Sarjapur Rd...",Coupon: Get 10% OFF. Use Code SUPER10 \n\n\n\...,"https://www.google.com/maps/place/12.9097955,7..."
...,...,...,...,...,...,...,...,...,...,...,...,...
404,New Year in the Silence of the Mountains,Uttarakhand,NCR,NamaStay Himalayas,"Concert, Outdoor Adventure, Nation Wide, Food ...","Saturday, 31 Dec 04:00 PM - Sunday, 01 Jan 12:...",2499 INR,Yudhish,-,"Bhairo Dhunga, Kanatal, Tehri Range, Uttarakha...",The event starts at 4:00 PM. Venue rules apply.,"https://www.google.com/maps/place/30.4272619,7..."
405,CrossOver On Sea | NYE 2023,Panaji,Goa,CrossOver On The Sea ( boarding point ),"Dj Night, EDM, Nation Wide, Techno, New Years ...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 04:...",3000 - 50000 INR,Centigradeevents,"Melocomp, Williamboy, Auroteq, Createawitty,","GR2G+F66, Betim Ferry, Panaji, Goa 403001, India",The event starts at 8:00 PM. Venue rules apply.,"https://www.google.com/maps/place/,"
406,Sacred New Year Countdown,Indore,NCR,Pride Hotel & Convention Centre Indore,"Dj Night, Bollywood Night, Exclusive Experienc...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 12:...",3500 - 40000 INR,Ritesh Jaiswal,"DJ Vaibhav,","35, PW5H+6VQ, 1, Bicholi Mardana, near pride h...",The event starts at 8:00 PM. Venue rules apply.,"https://www.google.com/maps/place/22.7080794,7..."
407,Cyber Pirates - New Year Blast,,Goa,Ocean Pirates Cafe,"Dj Night, EDM, New Years Eve 2023","Saturday, 31 Dec 06:00 PM - Sunday, 01 Jan 08:...",1000 INR,Workzone,"Flip Knot, Aztec Dosage, Fuzulu, Spiritual Gan...","799/1, Market Rd, Monteiro Vaddo, Anjuna, Goa ...",The event starts at 6:00 PM. Venue rules apply.,"https://www.google.com/maps/place/15.5783436,7..."


In [35]:
pa = "\d{2}[.]\d+[,]+\d{2}[.]\d+"
o = df["Googlemap Link"].iloc[1]
re.findall(pa,o)

['12.9572489,77.74502929999994']

In [5]:
def long_lat(data):
    x = data
    pattern = "\d{2}[.]\d+[,]+\d{2}[.]\d+"
    y = re.findall(pattern,x)
    if len(y) == 1:
        for items in y:
            return(items)
    elif len(y) == 0:
        return(x)

In [6]:
df["long_lat"] = df["Googlemap Link"].apply(long_lat)

In [7]:
df

,Title,Location,City,Venue,Highlights,Schedule,Price,Organised by,Artists,Address,Note,Googlemap Link,long_lat
0,Moonlit New Year Eve Party 2023,Whitefield,Bangalore,Virginia Mall,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 05:00 PM - Sunday, 01 Jan 12:...",499 - 1499 INR,Oxygen9events,"Krish Hart, DJ Prackze, DJ Prashanth, DJ Nitin...","Virginia Mall, Varthur Main Rd, Palm Meadows, ...",Coupon: Get 5% OFF. Use Code MOON5 \n\n\n\nTh...,"https://www.google.com/maps/place/12.9572489,7...","12.9572489,77.74502929999994"
1,New Year Bash 2023 At Nolimmits Lounge And Clu...,Brigade Road,Bangalore,NoLimmits Lounge and Club,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 08:30 PM - Sunday, 01 Jan 12:...",3150 - 7350 INR,Nolimmits Lounge & Club,"Dj Esha, DJ VCKY, DJ VeeJay,","Bengaluru, Ashok Nagar, Bangalore Urban, Karna...",Coupon: Get 10% OFF. Use Code NOLIMMITS10 \n\...,"https://www.google.com/maps/place/12.970303,77...","12.970303,77.610947"
2,Soiree NYE 2023,Whitefield,Bangalore,Bengaluru Marriott Hotel Whitefield,"Dj Night, Band Performance, Bollywood Night, F...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 01:...",2006 - 11799 INR,Marriot Bangalore,-,"75 EPIP Area, 8th Rd, EPIP Zone, Whitefield, B...",Coupon: Get 10% OFF. Use Code SOIREE10 \n\n\n...,"https://www.google.com/maps/place/12.979255,77...","12.979255,77.72794399999998"
3,Baadshaah NYE 2023 Carnival,marathahalli,Bangalore,The Big Baadshaah,"Dj Night, Bollywood Night, Food & Drinks, New ...","Saturday, 31 Dec 07:00 PM - Sunday, 01 Jan 12:...",2399 - 5999 INR,THE BIG BAADSHAAH,"DJ Savanth,","88, 2nd Floor, Dr Puneeth Rajkumar Rd, near Mo...",Coupon: Get 10% OFF. Use Code CARNIVAL10 \n\n...,"https://www.google.com/maps/place/12.9484311,7...","12.9484311,77.6987728"
4,Celestial Supernova - NYE 2023 (Sarjapur),Kaikondrahalli,Bangalore,Big Pitcher Sarjapur Road,"Dj Night, Bollywood Night, Concert, Offers, Fi...","Saturday, 31 Dec 06:30 PM - Sunday, 01 Jan 12:...",0 - 7488 INR,FOODALE INDIA PVT LTD,"DJ Rohith, Dj Shugi Born, DJ Pruthvi, DJ Tony,...","Doddakannelli, 116/9, Ambalipura - Sarjapur Rd...",Coupon: Get 10% OFF. Use Code SUPER10 \n\n\n\...,"https://www.google.com/maps/place/12.9097955,7...","12.9097955,77.6885204"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,New Year in the Silence of the Mountains,Uttarakhand,NCR,NamaStay Himalayas,"Concert, Outdoor Adventure, Nation Wide, Food ...","Saturday, 31 Dec 04:00 PM - Sunday, 01 Jan 12:...",2499 INR,Yudhish,-,"Bhairo Dhunga, Kanatal, Tehri Range, Uttarakha...",The event starts at 4:00 PM. Venue rules apply.,"https://www.google.com/maps/place/30.4272619,7...","30.4272619,78.33089729999999"
405,CrossOver On Sea | NYE 2023,Panaji,Goa,CrossOver On The Sea ( boarding point ),"Dj Night, EDM, Nation Wide, Techno, New Years ...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 04:...",3000 - 50000 INR,Centigradeevents,"Melocomp, Williamboy, Auroteq, Createawitty,","GR2G+F66, Betim Ferry, Panaji, Goa 403001, India",The event starts at 8:00 PM. Venue rules apply.,"https://www.google.com/maps/place/,","https://www.google.com/maps/place/,"
406,Sacred New Year Countdown,Indore,NCR,Pride Hotel & Convention Centre Indore,"Dj Night, Bollywood Night, Exclusive Experienc...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 12:...",3500 - 40000 INR,Ritesh Jaiswal,"DJ Vaibhav,","35, PW5H+6VQ, 1, Bicholi Mardana, near pride h...",The event starts at 8:00 PM. Venue rules apply.,"https://www.google.com/maps/place/22.7080794,7...","22.7080794,75.9297425"
407,Cyber Pirates - New Year Blast,,Goa,Ocean Pirates Cafe,"Dj Night, EDM, New Years Eve 2023","Saturday, 31 Dec 06:00 PM - Sunday, 01 Jan 08:...",1000 INR,Workzone,"Flip Knot, Aztec Dosage, Fuzulu, Spiritual Gan...","799/1, Market Rd, Monteiro Vaddo, Anjuna, Goa ...",The event starts at 6:00 PM. Venue rules apply.,"https://www.google.com/maps/place/15.5783436,7...","15.5783436,73.7392863"


In [8]:
df.drop(['Googlemap Link'], axis=1,inplace = True)


In [9]:
df

,Title,Location,City,Venue,Highlights,Schedule,Price,Organised by,Artists,Address,Note,long_lat
0,Moonlit New Year Eve Party 2023,Whitefield,Bangalore,Virginia Mall,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 05:00 PM - Sunday, 01 Jan 12:...",499 - 1499 INR,Oxygen9events,"Krish Hart, DJ Prackze, DJ Prashanth, DJ Nitin...","Virginia Mall, Varthur Main Rd, Palm Meadows, ...",Coupon: Get 5% OFF. Use Code MOON5 \n\n\n\nTh...,"12.9572489,77.74502929999994"
1,New Year Bash 2023 At Nolimmits Lounge And Clu...,Brigade Road,Bangalore,NoLimmits Lounge and Club,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 08:30 PM - Sunday, 01 Jan 12:...",3150 - 7350 INR,Nolimmits Lounge & Club,"Dj Esha, DJ VCKY, DJ VeeJay,","Bengaluru, Ashok Nagar, Bangalore Urban, Karna...",Coupon: Get 10% OFF. Use Code NOLIMMITS10 \n\...,"12.970303,77.610947"
2,Soiree NYE 2023,Whitefield,Bangalore,Bengaluru Marriott Hotel Whitefield,"Dj Night, Band Performance, Bollywood Night, F...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 01:...",2006 - 11799 INR,Marriot Bangalore,-,"75 EPIP Area, 8th Rd, EPIP Zone, Whitefield, B...",Coupon: Get 10% OFF. Use Code SOIREE10 \n\n\n...,"12.979255,77.72794399999998"
3,Baadshaah NYE 2023 Carnival,marathahalli,Bangalore,The Big Baadshaah,"Dj Night, Bollywood Night, Food & Drinks, New ...","Saturday, 31 Dec 07:00 PM - Sunday, 01 Jan 12:...",2399 - 5999 INR,THE BIG BAADSHAAH,"DJ Savanth,","88, 2nd Floor, Dr Puneeth Rajkumar Rd, near Mo...",Coupon: Get 10% OFF. Use Code CARNIVAL10 \n\n...,"12.9484311,77.6987728"
4,Celestial Supernova - NYE 2023 (Sarjapur),Kaikondrahalli,Bangalore,Big Pitcher Sarjapur Road,"Dj Night, Bollywood Night, Concert, Offers, Fi...","Saturday, 31 Dec 06:30 PM - Sunday, 01 Jan 12:...",0 - 7488 INR,FOODALE INDIA PVT LTD,"DJ Rohith, Dj Shugi Born, DJ Pruthvi, DJ Tony,...","Doddakannelli, 116/9, Ambalipura - Sarjapur Rd...",Coupon: Get 10% OFF. Use Code SUPER10 \n\n\n\...,"12.9097955,77.6885204"
...,...,...,...,...,...,...,...,...,...,...,...,...
404,New Year in the Silence of the Mountains,Uttarakhand,NCR,NamaStay Himalayas,"Concert, Outdoor Adventure, Nation Wide, Food ...","Saturday, 31 Dec 04:00 PM - Sunday, 01 Jan 12:...",2499 INR,Yudhish,-,"Bhairo Dhunga, Kanatal, Tehri Range, Uttarakha...",The event starts at 4:00 PM. Venue rules apply.,"30.4272619,78.33089729999999"
405,CrossOver On Sea | NYE 2023,Panaji,Goa,CrossOver On The Sea ( boarding point ),"Dj Night, EDM, Nation Wide, Techno, New Years ...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 04:...",3000 - 50000 INR,Centigradeevents,"Melocomp, Williamboy, Auroteq, Createawitty,","GR2G+F66, Betim Ferry, Panaji, Goa 403001, India",The event starts at 8:00 PM. Venue rules apply.,"https://www.google.com/maps/place/,"
406,Sacred New Year Countdown,Indore,NCR,Pride Hotel & Convention Centre Indore,"Dj Night, Bollywood Night, Exclusive Experienc...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 12:...",3500 - 40000 INR,Ritesh Jaiswal,"DJ Vaibhav,","35, PW5H+6VQ, 1, Bicholi Mardana, near pride h...",The event starts at 8:00 PM. Venue rules apply.,"22.7080794,75.9297425"
407,Cyber Pirates - New Year Blast,,Goa,Ocean Pirates Cafe,"Dj Night, EDM, New Years Eve 2023","Saturday, 31 Dec 06:00 PM - Sunday, 01 Jan 08:...",1000 INR,Workzone,"Flip Knot, Aztec Dosage, Fuzulu, Spiritual Gan...","799/1, Market Rd, Monteiro Vaddo, Anjuna, Goa ...",The event starts at 6:00 PM. Venue rules apply.,"15.5783436,73.7392863"


In [11]:
cleaned_df = df.drop_duplicates()

In [12]:
cleaned_df

,Title,Location,City,Venue,Highlights,Schedule,Price,Organised by,Artists,Address,Note,long_lat
0,Moonlit New Year Eve Party 2023,Whitefield,Bangalore,Virginia Mall,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 05:00 PM - Sunday, 01 Jan 12:...",499 - 1499 INR,Oxygen9events,"Krish Hart, DJ Prackze, DJ Prashanth, DJ Nitin...","Virginia Mall, Varthur Main Rd, Palm Meadows, ...",Coupon: Get 5% OFF. Use Code MOON5 \n\n\n\nTh...,"12.9572489,77.74502929999994"
1,New Year Bash 2023 At Nolimmits Lounge And Clu...,Brigade Road,Bangalore,NoLimmits Lounge and Club,"Dj Night, Bollywood Night, EDM, Food & Drinks,...","Saturday, 31 Dec 08:30 PM - Sunday, 01 Jan 12:...",3150 - 7350 INR,Nolimmits Lounge & Club,"Dj Esha, DJ VCKY, DJ VeeJay,","Bengaluru, Ashok Nagar, Bangalore Urban, Karna...",Coupon: Get 10% OFF. Use Code NOLIMMITS10 \n\...,"12.970303,77.610947"
2,Soiree NYE 2023,Whitefield,Bangalore,Bengaluru Marriott Hotel Whitefield,"Dj Night, Band Performance, Bollywood Night, F...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 01:...",2006 - 11799 INR,Marriot Bangalore,-,"75 EPIP Area, 8th Rd, EPIP Zone, Whitefield, B...",Coupon: Get 10% OFF. Use Code SOIREE10 \n\n\n...,"12.979255,77.72794399999998"
3,Baadshaah NYE 2023 Carnival,marathahalli,Bangalore,The Big Baadshaah,"Dj Night, Bollywood Night, Food & Drinks, New ...","Saturday, 31 Dec 07:00 PM - Sunday, 01 Jan 12:...",2399 - 5999 INR,THE BIG BAADSHAAH,"DJ Savanth,","88, 2nd Floor, Dr Puneeth Rajkumar Rd, near Mo...",Coupon: Get 10% OFF. Use Code CARNIVAL10 \n\n...,"12.9484311,77.6987728"
4,Celestial Supernova - NYE 2023 (Sarjapur),Kaikondrahalli,Bangalore,Big Pitcher Sarjapur Road,"Dj Night, Bollywood Night, Concert, Offers, Fi...","Saturday, 31 Dec 06:30 PM - Sunday, 01 Jan 12:...",0 - 7488 INR,FOODALE INDIA PVT LTD,"DJ Rohith, Dj Shugi Born, DJ Pruthvi, DJ Tony,...","Doddakannelli, 116/9, Ambalipura - Sarjapur Rd...",Coupon: Get 10% OFF. Use Code SUPER10 \n\n\n\...,"12.9097955,77.6885204"
...,...,...,...,...,...,...,...,...,...,...,...,...
315,Dubai Edition New Year 2023 | NYE 2023,Kondhwa,Pune,BETOS' S Bar and Kitchen,"Dj Night, Bollywood Night, Exclusive Experienc...","Saturday, 31 Dec 05:00 PM - Sunday, 01 Jan 01:...",1500 - 31000 INR,Rupesh Betos Bar,-,"2nd Runwal Diamond, NIBM Rd, opp. Palace Orcha...",The event starts at 5:00 PM. Venue rules apply.,"18.4637597,73.9046159"
316,Pune's Biggest Bollywood NYE 2023,Kharadi,Pune,Mayfield Estate,"Dj Night, Bollywood Night, Food & Drinks, New ...","Monday, 26 Dec 12:00 AM - Saturday, 31 Dec 08:...",999 - 64999 INR,Avm,"DJ Organic,","near Mio pallazio society, Tulaja Bhawani Naga...",The event starts at 12:00 AM. Venue rules apply.,"18.5605547,73.9504669"
317,NYE Bash 2023 | Win a Trip to Dubai & Enjoy,Hadapsar,Pune,The cult,"Dj Night, Bollywood Night, Food & Drinks, New ...","Saturday, 31 Dec 07:00 PM - Sunday, 01 Jan 01:...",1499 - 100000 INR,The Cult House,"Rohit, LPK Goa, DJ Abhishek Mantri,","Kad Wasti, Magarpatta, Hadapsar, Pune, Maharas...",The event starts at 7:00 PM. Venue rules apply.,"18.5200411,73.92003339999997"
347,Zone Out - NYE,Morjim Beach Road,Goa,Pappi Chulo Morjim,"Dj Night, Concert, EDM, Food & Drinks, Techno,...","Saturday, 31 Dec 08:00 PM - Sunday, 01 Jan 01:...",Free Entry,Smash Entertainment,"Danique, Dobrov, Avatar, Swap, Zia,","Morjim Beach Road, Morjim, Goa 403512, India",The event starts at 8:00 PM. Venue rules apply.,"15.6263875,73.74209689999999"


In [20]:
cleaned_df.to_excel("event_data.xlsx")

In [21]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262 entries, 0 to 348
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         262 non-null    object
 1   Location      262 non-null    object
 2   City          262 non-null    object
 3   Venue         262 non-null    object
 4   Highlights    262 non-null    object
 5   Schedule      262 non-null    object
 6   Price         262 non-null    object
 7   Organised by  262 non-null    object
 8   Artists       262 non-null    object
 9   Address       262 non-null    object
 10  Note          262 non-null    object
 11  long_lat      262 non-null    object
dtypes: object(12)
memory usage: 26.6+ KB
